# Расчет дифференциальных уравнений на графах

## Задача 1. Моделирование транспортных потоков

### Часть 1. Модель транспортного потока

Для моделирования транспортных потоков будем использовать макроскопический подход.

Уравнения, описывающие поведение потока на автостраде:
$$
\begin{cases}
    & \rho_t + (\rho u)_x = 0 \\
    & (\rho \omega)_t + (\rho u \omega)_x = 0 \\
    & u = V(\rho, \omega)
\end{cases}
$$

Фундаментальная диаграмма задается как:
$$
\begin{align*}
    & Q_{\alpha, \lambda, p}(\rho) = \alpha \left(
        a + (b-a) \frac{\rho}{\rho_{max}} - \sqrt{1 + y^2}
    \right) \\
    & a = \sqrt{1 + (\lambda p)^2} \\
    & b = \sqrt{1 + (\lambda (1 - p) )^2} \\
    & y = \lambda \left( \frac{\rho}{\rho_{max}} - p \right)
\end{align*}
$$
Положим, что параметры фундаментальной диаграммы зависят от $\omega$:
$$
\begin{align*}
    & \alpha = \alpha(\omega)
    & \lambda = \lambda(\omega)
    & p = p(\omega)
\end{align*}
$$
Алгоритм определения этой зависимсоти будет дан позже.

Скорость потока задается следующим образом:
$$
V(\rho, \omega) = \begin{cases}
    & \frac{Q(\rho, \omega)}{\rho} & ,\rho > 0 \\
    & Q_\rho(0, \omega) & ,\rho = 0
\end{cases}
$$

### Часть 2. Разностная схема

Сделаем замену $y := \rho \omega$.
Тогда система уравнений может быть переписана в виде:
$$
\begin{cases}
    & \rho_t + (\rho u)_x = 0 \\
    & y_t + (u y)_x = 0 \\
    & u = V(\rho, y/\rho)
\end{cases}
$$

Обозначим $\mathbf{U} = (\rho, y)$ и $\mathbf{F}(\mathbf{U}) = (\rho u, u y)$. <br>
Тогда уравнения можно записать так:
$$
\mathbf{U}_t + \mathbf{F}_x(\mathbf{U}) = 0
$$

Для численного приближения данной системы будем использовать метод Годунова.
Основная идея в применении следующей аппроксимации:
$$
\mathbf{U}^n_i \approx \frac{1}{\Delta x} \int\limits_{x_{i - 1/2}}^{x_{i + 1/2}} \mathbf{U}(x, t_n) dx
$$

Конечно-разностная аппроксимация исходных уравнений выглядит как:
$$
\mathbf{U}^{n+1}_i = \mathbf{U}^{n}_i - \frac{\Delta t}{\Delta x} \left(
    \mathbf{F}^n_{i+1/2} - \mathbf{F}^n_{i-1/2}
\right)
$$

Где значения потока $\mathbf{F}$ в полуцелых узлах записывается как:
$$
\begin{align*}
    & \mathbf{F}^n_{i + 1/2} = \mathbf{F}( \mathcal{R}( \mathbf{U}^n_{i} , \mathbf{U}^n_{i+1} ) ) \\
    & \mathbf{F}^n_{i - 1/2} = \mathbf{F}( \mathcal{R}( \mathbf{U}^n_{i-1} , \mathbf{U}^n_{i} ) )
\end{align*}
$$
Где $\mathcal{R}(\mathbf{U}_L, \mathbf{U}_R)$ обозначает решение задачи Римана для начальных состояний $\mathbf{U}_L$  и $\mathbf{U}_R$

Перепишем эти уравнения покомпонентно:
$$
\begin{align*}
    & \rho^{n+1}_j = \rho^{n}_j - \frac{\Delta t}{\Delta x} \left( Q^{n}_{j+1/2} - Q^{n}_{j-1/2} \right) \\
    & y^{n+1}_j = y^{n}_j - \frac{\Delta t}{\Delta x} \left( \omega^n_{j} Q^{n}_{j+1/2} - \omega^n_{j-1} Q^{n}_{j-1/2} \right)
\end{align*}
$$
Где $\omega^n_j = y^n_j / \rho^n_j$

Опишем решение задачи Римана.<br>
"Входной" и "выходной" потоки обозначим как $\mathbf{U}_L = (\rho_L, \rho_L \omega_L)$ и $\mathbf{U}_R = (\rho_R, \rho_R \omega_R)$ соответственно.

Используемое нами решение основано на функциях "отправки" и "получения":
$$
\begin{align*}
    & S(\rho_L, u_L, \omega_L) = \begin{cases}
        & \rho_L u_L 
        &,\rho_L \leq \rho_c(\omega_L) \\
        & Q^{max}_{\omega_L}
        &,\rho_L > \rho_c(\omega_L)
    \end{cases} \\
    & R(\rho_M, u_M, \omega_L) = \begin{cases}
        & Q^{max}_{\omega_L}
        &,\rho_M \leq \rho_c(\omega_L) \\
        & \rho_M u_M
        &,\rho_M > \rho_c(\omega_L)
    \end{cases}
\end{align*}
$$
Где $\rho_c(\omega_L)$ критическая плотность при заданном параметре $\omega_L$.

Промежуточные значения плотности и скорости задаются как:
$$
\begin{align*}
    & \rho_M(\omega_L, u_R) = \mathrm{argmin}_\rho \left( u_R - V(\rho, \omega_L)\right) \\
    & u_M(\omega_L, u_R) = \min \left\{ V(0,\omega_L), u_R \right\}
\end{align*}
$$

Таким образом, промежуточные значения потока вычисляются следующим образом:
$$
\mathcal{R}(\mathbf{U}_L, \mathbf{U}_R) = \min \left\{
    S(\rho_L, u_L, \omega_L), R(\rho_M, u_M, \omega_L) 
\right\}
$$

### Часть 3. Алгоритм

Опишем алгоритм:
1. Находим параметр $\omega$:
    $$
    \omega^n_j = \frac{y^n_j}{\rho^n_j}
    $$
    Где $j=1,\ldots,N$
1. Находим скорости в узлах:
    $$
        u^n_{j} = V(\rho^n_j, \omega^n_j)
    $$
    Где $j=1 \ldots, N$
1. Находим промежуточные скорости:
    $$
    \begin{align*}
        & u^n_{j-1/2} = \min \left\{ V(0, \omega^n_{j-1}), u^n_{j}) \right\}
    \end{align*}
    $$
    Где $j=2, \dots, N$
1. Находим промежуточные плотности:
    $$
    \begin{align*}
        & \rho^n_{j-1/2} = \mathrm{argmin}_{\rho} \left( u^n_j - V(\rho, \omega^n_{j-1}) \right)
    \end{align*}
    $$
    Где $j=2, \ldots, N$

2. Находим функции отправки и получения:
    $$
    \begin{align*}
        & S^n_{j-1/2} = S(\rho^n_{j-1}, u^n_{j-1}, \omega^n_{j-1}) \\
        & R^n_{j-1/2} = R(\rho^n_{j-1/2}, u^n_{j-1/2}, \omega^n_{j-1})
    \end{align*}
    $$
    Где $j=2,\ldots,N$
    
6. Учитываем граничные условия посредством функций отправки и получения:
    $$
    \begin{align*}
        & S^n_{1/2}, R^n_{1/2} \text{-- левая граница} \\
        & S^n_{N+1/2}, R^n_{N+1/2} \text{-- правая граница}
    \end{align*}
    $$

5. Находим интенсивность потока:
    $$
    Q^n_{j-1/2} = \min \left\{ S^n_{j-1/2}, R^n_{j-1/2} \right\}
    $$
    Где $j=1,\ldots,N+1$

6. Находим состояние системы на следующем временном шаге:
    $$
    \begin{align*}
        & \rho^{n+1}_j = \rho^{n}_j - \frac{\Delta t}{\Delta x} \left( Q^{n}_{j+1/2} - Q^{n}_{j-1/2} \right) \\
        & y^{n+1}_j = y^{n}_j - \frac{\Delta t}{\Delta x} \left( \omega^n_{j} Q^{n}_{j+1/2} - \omega^n_{j-1} Q^{n}_{j-1/2} \right)
    \end{align*}
    $$
    Где $j=1,\ldots,N$